## Imports

In [ ]:
import pickle
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import ProcessData as dp
import VisLib as vis

## Get Data Loaders

In [ ]:
with open('goodDataLoaders', 'rb') as filehandle:
    # read the data as binary data stream
    dataLoaders = pickle.load(filehandle)
trainLoader = dataLoaders[0] 
evalLoader = dataLoaders[1]
testLoader = dataLoaders[2]

## Define Model

In [ ]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
        
#         self.conv1 = nn.Conv2d(6,16,kernel_size=(1,3),stride=1,padding=(0,1))
#         self.conv2 = nn.Conv2d(16,32,kernel_size=(1,3),stride=1,padding=(0,1))
#         self.fc1 = nn.Linear(2113, 128)
#         self.fc2 = nn.Linear(128, 8)
#         self.dropout = nn.Dropout(p=0.5)
#         self.pool = nn.MaxPool2d((1, 3), stride=1)    
    

#     def forward(self, x, f):
#         x.view(x.size(0), -1)
#         x = self.dropout(self.pool(F.relu(self.conv1(x))))
#         x = self.dropout(self.pool(F.relu(self.conv2(x))))
#         x = x.view(x.size(0), -1)   
#         x = torch.cat((x,f), dim=1)
#         x = self.dropout(F.relu(self.fc1(x)))
#         x = self.fc2(x)
        
#         return x

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(6,24,kernel_size=(1,5),stride=1,padding=(0,1))
        self.conv2 = nn.Conv2d(24,24, kernel_size=(1,5),stride=1,padding=(0,1))
        self.conv3 = nn.Conv2d(24,24, kernel_size=(1,5),stride=1,padding=(0,1))
        self.conv4 = nn.Conv2d(24,24, kernel_size=(1,5),stride=1,padding=(0,1))
        self.fc1 = nn.Linear(1401-30, 50)
        self.fc2 = nn.Linear(50, 8)
        self.dropout = nn.Dropout(p=0.5)
        self.pool = nn.MaxPool2d((1, 2), stride=1)

    def forward(self, x, f):
        x.view(x.size(0), -1)
        x = F.relu( self.conv1(x) )
        x = self.pool(x)
        x = self.dropout(x)
        x = F.relu( self.conv2(x) )
        x = self.pool(x)
        x = self.dropout(x)
        x = F.relu( self.conv3(x) )
        x = self.pool(x)
        x = self.dropout(x)
        x = F.relu( self.conv4(x) )
        x = self.pool(x)
        x = self.dropout(x)
        x = x.view(x.size(0), -1)

        x = torch.cat((x,f[:,[2,3,5]]), dim=1)
        
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x


net = Net()


## Setup for Training

In [ ]:
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_uniform_(m.weight.data)
        nn.init.xavier_uniform_(m.weight.data)
        
def weight_reset(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        m.reset_parameters()

net = Net()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

criterion = nn.CrossEntropyLoss()
learning_rate = 1e-4
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)


# net.apply(weights_init)

## Train Net

In [ ]:
train_loss_values = []
train_terrain_accuracy = []
train_class_accuracy = []

val_terrain_accuracy = []
val_class_accuracy = []

best_acc = 0
terrain_flag = False
class_flag = True

model_path = './6_11_complex_allFeatures_class'
curve_data_filename = '6_11_curveData_complex_allFeatures_class'

for epoch in range(1000):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainLoader, 0):

        net = net.train()

        inputs = data['tactile_data']
        features = data['custom_features']
        labels_class = data['class']
        labels_terrain = data['terrain']

        inputs = inputs.float()
        features = features.float()
        labels_class = labels_class.long()
        labels_terrain = labels_terrain.long()

        inputs = inputs.to(device)
        features = features.to(device)
        labels_class = labels_class.to(device)
        labels_terrain = labels_terrain.to(device)

        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs, features)
        
        if terrain_flag:
            loss = criterion(outputs, labels_terrain)
        elif class_flag:
            loss = criterion(outputs, labels_class)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()


        if i % 2000 == 1999:    # print every 2000 mini-batches

            net = net.eval()

            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            train_loss_values.append(running_loss)

            train_acc = vis.GetAcc(net, epoch, trainLoader, 'train', terrain_flag=terrain_flag)
            val_acc = vis.GetAcc(net, epoch, evalLoader, 'eval', terrain_flag=terrain_flag)

            if terrain_flag:
                train_terrain_accuracy.append(train_acc)
                val_terrain_accuracy.append(val_acc)
            elif class_flag:
                train_class_accuracy.append(train_acc)
                val_class_accuracy.append(val_acc)

            if val_acc > best_acc:
                PATH = model_path
                torch.save(net.state_dict(), PATH)
                best_acc = val_acc
                if terrain_flag:
                    curve_data = [train_loss_values, train_terrain_accuracy, val_terrain_accuracy]
                elif class_flag:
                    curve_data = [train_loss_values, train_class_accuracy, val_class_accuracy]
                with open(curve_data_filename, 'wb') as filehandle:
                    pickle.dump(curve_data, filehandle)
                print('New Best Accuracy: %.3f' % best_acc)


            running_loss = 0.0

print('Finished Training')

## Plot Results

In [ ]:
model = Net()
model.load_state_dict(torch.load('./6_11_complex_gaitParam_terrain'))
model.cuda()

cdt, idt, cdc, idc = vis.GetDicts(model, evalLoader, terrain_flag = True)
# cdc, idc = vis.GetDicts(model, evalLoader, terrain_flag = False)

vis.PlotConfusionMatrix(vis.CreateConfusionMatrix(cdt, idt), vis.terrain_labels)
# vis.PlotConfusionMatrix(vis.CreateConfusionMatrix(cdc, idc), vis.class_labels)

In [ ]:
with open('6_11_curveData_complex_gaitParam_terrain', 'rb') as filehandle:
    # read the data as binary data stream
    curve_data = pickle.load(filehandle)
tlv = curve_data[0] 
tca = curve_data[1]
vca = curve_data[2]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

num_epochs = len(tca)/2 # because it saves a point twive per epoch
training_time = np.linspace(0, num_epochs, len(tca))

plt.rcParams.update({'font.size': 22})
plt.plot(np.array(training_time), np.array(tlv), 'r')
plt.xlabel('training time')
plt.ylabel('training loss')
plt.show()

plt.plot(np.array(training_time), np.array(tca), 'b', label="Train")
plt.plot(np.array(training_time), np.array(vca), 'g', label="Validation")
plt.xlabel('training epoch')
plt.ylabel('terrain accuracy [%]')
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0.)
plt.show()